In [1]:
import os
import csv
import random
from glob import glob
from tqdm import tqdm
from collections import namedtuple

import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch

import sys
sys.path.append("/kaggle/input/super-glue-pretrained-network")
from models.matching import Matching
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

dry_run = False

In [2]:
src = '/kaggle/input/image-matching-challenge-2022/'

test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]

if dry_run:
    for sample in test_samples:
        print(sample)
        
test_samples_df = pd.DataFrame(test_samples, columns=["sample_id", "batch_id", "image_1_id", "image_2_id"])
test_samples_df

,sample_id,batch_id,image_1_id,image_2_id
0,googleurban;1cf87530;a5a9975574c94ff9a285f58c3...,1cf87530,a5a9975574c94ff9a285f58c39b53d2c,0143f47ee9e54243a1b8454f3e91621a
1,googleurban;6ceaefff;39563e58b2b7411da3f06427c...,6ceaefff,39563e58b2b7411da3f06427c9ee4239,0303b05ca0cb46959eac430e4b2472ca
2,googleurban;d91db836;81dd07fb7b9a4e01996cee637...,d91db836,81dd07fb7b9a4e01996cee637f91ca1a,0006b1337a0347f49b4e651c035dfa0e


# SuperPoint/SuperGlue Inference

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
resize = [-1, ]
resize_float = True

config = {
    "superpoint": {
        "nms_radius": 4,
        "keypoint_threshold": 0.005,
        "max_keypoints": 1024
    },
    "superglue": {
        "weights": "outdoor",
        "sinkhorn_iterations": 20,
        "match_threshold": 0.2,
    }
}
matching = Matching(config).eval().to(device)

Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


In [4]:
F_dict = {}
for i, row in tqdm(enumerate(test_samples)):
    sample_id, batch_id, image_1_id, image_2_id = row
    
    image_fpath_1 = f'{src}/test_images/{batch_id}/{image_1_id}.png'
    image_fpath_2 = f'{src}/test_images/{batch_id}/{image_2_id}.png'
    
    image_1, inp_1, scales_1 = read_image(image_fpath_1, device, resize, 0, resize_float)
    image_2, inp_2, scales_2 = read_image(image_fpath_2, device, resize, 0, resize_float)
    
    pred = matching({"image0": inp_1, "image1": inp_2})
    pred = {k: v[0].detach().cpu().numpy() for k, v in pred.items()}
    kpts1, kpts2 = pred["keypoints0"], pred["keypoints1"]
    matches, conf = pred["matches0"], pred["matching_scores0"]

    valid = matches > -1
    mkpts1 = kpts1[valid]
    mkpts2 = kpts2[matches[valid]]
    mconf = conf[valid]
    
    if len(mkpts1) > 8:
        F, inlier_mask = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_MAGSAC, ransacReprojThreshold=0.25, confidence=0.99999, maxIters=10000)
        F_dict[sample_id] = F
    else:
        F_dict[sample_id] = np.zeros((3, 3))

3it [00:05,  1.98s/it]


In [5]:
sg_matrix = list(F_dict.values()) #[ 8.49077204e-08,  1.59295878e-06, -7.60297330e-04]
sg_matrix

[array([[ 8.49077204e-08,  1.59295878e-06, -7.60297330e-04],
        [ 2.15347610e-06,  3.73637850e-08,  9.14656017e-03],
        [-1.53669112e-03, -1.00098670e-02,  5.04602901e-01]]),
 array([[-1.61503391e-06,  1.45412113e-05, -8.66035536e-03],
        [-1.66341233e-05,  4.26469564e-06,  1.23128804e-02],
        [ 7.46974807e-03, -1.59685012e-02,  4.27513386e+00]]),
 array([[-9.19709363e-07, -1.39336628e-05,  9.37422553e-03],
        [ 1.10619598e-05,  2.65573084e-05, -6.30746753e-03],
        [-6.37339619e-03, -1.83202112e-02,  5.91271213e+00]])]

In [6]:
def FlattenMatrix(M, num_digits=8):
    '''Convenience function to write CSV files.'''
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

#with open('sp_sg.csv', 'w') as f:
    #f.write('sample_id,fundamental_matrix\n')
    #for sample_id, F in F_dict.items():
        #f.write(f'{sample_id},{FlattenMatrix(F)}\n')

# DKM Inference

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys, os, csv
from PIL import Image
import cv2, gc
import matplotlib.pyplot as plt
import torch
sys.path.append('/kaggle/input/imc2022-dependencies/DKM/')

dry_run = False

In [8]:
os.getcwd()

'/kaggle/working'

In [9]:
!mkdir -p pretrained/checkpoints
!cp /kaggle/input/imc2022-dependencies/pretrained/dkm.pth pretrained/checkpoints/dkm_base_v11.pth

!pip install -f /kaggle/input/imc2022-dependencies/wheels --no-index einops
!cp -r /kaggle/input/imc2022-dependencies/DKM/ /kaggle/working/DKM/
!cd /kaggle/working/DKM/; pip install -f /kaggle/input/imc2022-dependencies/wheels -e .

Looking in links: /kaggle/input/imc2022-dependencies/wheels
Processing /kaggle/input/imc2022-dependencies/wheels/einops-0.4.1-py3-none-any.whl
Looking in links: /kaggle/input/imc2022-dependencies/wheels
Obtaining file:///kaggle/working/DKM
  Preparing metadata (setup.py) ... - done
  Running setup.py develop for dkm


In [10]:
if not torch.cuda.is_available():
    print('You may want to enable the GPU switch?')
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.hub.set_dir('/kaggle/working/pretrained/')

from dkm import dkm_base
model = dkm_base(pretrained=True, version="v11").to(device).eval()

In [11]:
F_dict = {}
for i, row in enumerate(test_samples):
    sample_id, batch_id, image_1_id, image_2_id = row

    img1 = cv2.imread(f'{src}/test_images/{batch_id}/{image_1_id}.png') 
    img2 = cv2.imread(f'{src}/test_images/{batch_id}/{image_2_id}.png')
        
    img1PIL = Image.fromarray(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    img2PIL = Image.fromarray(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    
    dense_matches, dense_certainty = model.match(img1PIL, img2PIL)
    dense_certainty = dense_certainty.sqrt()
    sparse_matches, sparse_certainty = model.sample(dense_matches, dense_certainty, 2000)
    
    mkps1 = sparse_matches[:, :2]
    mkps2 = sparse_matches[:, 2:]
    
    h, w, c = img1.shape
    mkps1[:, 0] = ((mkps1[:, 0] + 1)/2) * w
    mkps1[:, 1] = ((mkps1[:, 1] + 1)/2) * h

    h, w, c = img2.shape
    mkps2[:, 0] = ((mkps2[:, 0] + 1)/2) * w
    mkps2[:, 1] = ((mkps2[:, 1] + 1)/2) * h

    F, mask = cv2.findFundamentalMat(mkps1, mkps2, cv2.USAC_MAGSAC, 0.3, 0.9999, 25_000)

    
    good = F is not None and F.shape == (3,3)
    
    if good:
        F_dict[sample_id] = F
    else:
        F_dict[sample_id] = np.zeros((3, 3))
        continue

    gc.collect()
    
#with open('dkm.csv', 'w') as f:
    #f.write('sample_id,fundamental_matrix\n')
    #for sample_id, F in F_dict.items():
        #f.write(f'{sample_id},{FlattenMatrix(F)}\n')

In [12]:
dkm_matrix = list(F_dict.values()) #[ 8.97546963e-08,  1.36522772e-06, -6.21684625e-04]
dkm_matrix

[array([[ 9.49121822e-08,  1.40773113e-06, -6.36928500e-04],
        [ 1.54526073e-06, -4.68413765e-08,  7.95498862e-03],
        [-1.25832065e-03, -8.59906771e-03,  4.13876288e-01]]),
 array([[ 1.48790440e-06, -1.32801337e-05,  8.04295628e-03],
        [ 1.54911186e-05, -4.59025515e-06, -1.20560261e-02],
        [-7.26164195e-03,  1.60895309e-02, -4.33677029e+00]]),
 array([[ 2.41794269e-05,  3.39733886e-05, -2.89382434e-02],
        [-4.72904535e-05, -1.52090010e-06,  1.03649734e-02],
        [ 2.99958542e-02, -2.75752115e-02,  1.37488443e+01]])]

# LOFTR Inference

In [13]:
dry_run = False
!pip install ../input/loftrkornia/LOFTR-kornia/kornia-0.6.4-py2.py3-none-any.whl
!pip install ../input/loftrkornia/LOFTR-kornia/kornia_moons-0.1.9-py3-none-any.whl

Processing /kaggle/input/loftrkornia/LOFTR-kornia/kornia-0.6.4-py2.py3-none-any.whl
  Attempting uninstall: kornia
    Found existing installation: kornia 0.5.8
    Uninstalling kornia-0.5.8:
      Successfully uninstalled kornia-0.5.8
Processing /kaggle/input/loftrkornia/LOFTR-kornia/kornia_moons-0.1.9-py3-none-any.whl


In [14]:
import os
import numpy as np
import cv2
import csv
from glob import glob
import torch
import matplotlib.pyplot as plt
import kornia
from kornia_moons.feature import *
import kornia as K
import kornia.feature as KF
import gc

In [15]:
device = torch.device('cuda')
matcher = KF.LoFTR(pretrained=None)
matcher.load_state_dict(torch.load("../input/loftrkornia/LOFTR-kornia/loftr_outdoor.ckpt/loftr_outdoor.ckpt")['state_dict'])
matcher = matcher.to(device).eval()

In [16]:
def load_torch_image(fname, device):
    img = cv2.imread(fname)
    scale = 840 / max(img.shape[0], img.shape[1]) 
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    img = cv2.resize(img, (w, h))
    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img.to(device)

In [17]:
F_dict = {}
import time
for i, row in enumerate(test_samples):
    sample_id, batch_id, image_1_id, image_2_id = row
    # Load the images.
    st = time.time()
    image_1 = load_torch_image(f'{src}/test_images/{batch_id}/{image_1_id}.png', device)
    image_2 = load_torch_image(f'{src}/test_images/{batch_id}/{image_2_id}.png', device)
    print(image_1.shape)
    input_dict = {"image0": K.color.rgb_to_grayscale(image_1), 
              "image1": K.color.rgb_to_grayscale(image_2)}

    with torch.no_grad():
        correspondences = matcher(input_dict)
        
    mkpts0 = correspondences['keypoints0'].cpu().numpy()
    mkpts1 = correspondences['keypoints1'].cpu().numpy()
    
    if len(mkpts0) > 7:
        F, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.1800, 0.9999, 250000)
        inliers = inliers > 0
        assert F.shape == (3, 3), 'Malformed F?'
        F_dict[sample_id] = F
    else:
        F_dict[sample_id] = np.zeros((3, 3))
        continue
    gc.collect()
    nd = time.time()    
    if (i < 3):
        print("Running time: ", nd - st, " s")
        draw_LAF_matches(
        KF.laf_from_center_scale_ori(torch.from_numpy(mkpts0).view(1,-1, 2),
                                    torch.ones(mkpts0.shape[0]).view(1,-1, 1, 1),
                                    torch.ones(mkpts0.shape[0]).view(1,-1, 1)),

        KF.laf_from_center_scale_ori(torch.from_numpy(mkpts1).view(1,-1, 2),
                                    torch.ones(mkpts1.shape[0]).view(1,-1, 1, 1),
                                    torch.ones(mkpts1.shape[0]).view(1,-1, 1)),
        torch.arange(mkpts0.shape[0]).view(-1,1).repeat(1,2),
        K.tensor_to_image(image_1),
        K.tensor_to_image(image_2),
        inliers,
        draw_dict={'inlier_color': (0.2, 1, 0.2),
                   'tentative_color': None, 
                   'feature_color': (0.2, 0.5, 1), 'vertical': False})
    
#with open('submission.csv', 'w') as f:
    #f.write('sample_id,fundamental_matrix\n')
    #for sample_id, F in F_dict.items():
        #f.write(f'{sample_id},{FlattenMatrix(F)}\n')

torch.Size([1, 3, 840, 472])


/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/BinaryOps.cpp:461.)
  return torch.floor_divide(self, other)


Running time:  1.0458638668060303  s
torch.Size([1, 3, 840, 472])
Running time:  0.8402094841003418  s
torch.Size([1, 3, 840, 472])
Running time:  1.3170323371887207  s


In [18]:
loftr_matrix = list(F_dict.values()) #[ 1.33081005e-07,  1.89142917e-06, -8.26164486e-04]
loftr_matrix

[array([[ 1.33081005e-07,  1.89142917e-06, -8.26164486e-04],
        [ 1.21548603e-06, -1.69887159e-07,  8.47602404e-03],
        [-1.15330093e-03, -9.03091996e-03,  3.99727161e-01]]),
 array([[-1.41077929e-06,  1.25641038e-05, -7.69075050e-03],
        [-1.47192483e-05,  4.66520159e-06,  1.17920640e-02],
        [ 7.12669880e-03, -1.59184362e-02,  4.23460368e+00]]),
 array([[-2.10167616e-06, -2.33597415e-05,  1.33922129e-02],
        [ 1.56584782e-06, -3.03106291e-06,  8.26859973e-03],
        [-1.25428306e-03, -3.63535567e-03, -1.20946245e+00]])]

# Ensemble 

In [19]:
sg = np.vstack(sg_matrix)
dkm = np.vstack(dkm_matrix)
loftr = np.vstack(loftr_matrix)

In [20]:
#sg = sg * 0.3
#dkm = dkm * 0.2
#loftr = loftr * 0.5

In [21]:
av = np.add(sg, dkm, loftr)

In [22]:
av = av / 3

In [23]:
av = np.split(av,3)

In [24]:
for k,v in zip(F_dict,av):
    F_dict[k] = v

In [25]:
F_dict

{'googleurban;1cf87530;a5a9975574c94ff9a285f58c39b53d2c-0143f47ee9e54243a1b8454f3e91621a': array([[ 5.99399675e-08,  1.00022997e-06, -4.65741943e-04],
        [ 1.23291228e-06, -3.15919718e-09,  5.70051626e-03],
        [-9.31670590e-04, -6.20297825e-03,  3.06159730e-01]]),
 'googleurban;6ceaefff;39563e58b2b7411da3f06427c9ee4239-0303b05ca0cb46959eac430e4b2472ca': array([[-4.23765006e-08,  4.20359200e-07, -2.05799696e-04],
        [-3.81001576e-07, -1.08519836e-07,  8.56180944e-05],
        [ 6.93687086e-05,  4.03432248e-05, -2.05454774e-02]]),
 'googleurban;d91db836;81dd07fb7b9a4e01996cee637f91ca1a-0006b1337a0347f49b4e651c035dfa0e': array([[ 7.75323919e-06,  6.67990861e-06, -6.52133929e-03],
        [-1.20761646e-05,  8.34546943e-06,  1.35250194e-03],
        [ 7.87415266e-03, -1.52984742e-02,  6.55385216e+00]])}

In [26]:
with open('submission.csv', 'w') as f:
    f.write('sample_id,fundamental_matrix\n')
    for sample_id, F in F_dict.items():
        f.write(f'{sample_id},{FlattenMatrix(F)}\n')